In [2]:
#! pip install -r requirements.txt

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import MultiLabelBinarizer
from scipy import sparse
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import torch

In [ ]:
"""
from huggingface_hub import upload_folder

upload_folder(
    folder_path="app/model_assets",  # your local folder path
    repo_id="MohammedZien/hybrid-news-recommender-assets",
    repo_type="model",  # or "dataset"
    token=True
)
"""

f:\projects\Porfolio\recommender-system\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
behaviors.tsv:   0%|          | 0.00/92.0M [00:00<?, ?B/s]









behaviors.tsv:   0%|          | 131k/92.0M [00:00<01:41, 904kB/s]




behaviors.tsv:   0%|          | 229k/92.0M [00:00<02:10, 706kB/s]
behaviors.tsv:   0%|          | 410k/92.0M [00:00<02:59, 511kB/s]


behaviors.tsv:   1%|          | 492k/92.0M [00:01<04:11, 364kB/s]





behaviors.tsv:   1%|          | 557k/92.0M [00:01<05:22, 283kB/s]


behaviors.tsv:   1%|          | 623k/92.0M [00:01<05:03, 301kB/s]
behaviors.tsv:   1%|          | 688k/92.0M [00:01<04:30, 337kB/s]




behaviors.tsv:   1%|          | 737k/92.0M [00:01<04:35, 331kB/s]
behaviors.tsv:   1%|          | 786k/92.0M [00:02<04:14, 359kB/s]






behaviors.tsv:   1%|          | 885k/92.0M [00

CommitInfo(commit_url='https://huggingface.co/MohammedZien/hybrid-news-recommender-assets/commit/1f6bb7f9dc95ac8fde65f9c181e36128c3195e9f', commit_message='Upload folder using huggingface_hub', commit_description='', oid='1f6bb7f9dc95ac8fde65f9c181e36128c3195e9f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/MohammedZien/hybrid-news-recommender-assets', endpoint='https://huggingface.co', repo_type='model', repo_id='MohammedZien/hybrid-news-recommender-assets'), pr_revision=None, pr_num=None)

In [142]:
raw_news_data = pd.read_csv("F:/projects/Porfolio/recommender-system/app/model_assets/news.tsv",delimiter="\t",
                             names=["News ID" , "Category" , "Subcategory" , "News Title" ,"News Abstract", "News Url", "Entities in News Title", "Entities in News Abstract"],
                             index_col="News ID")

raw_news_data.sort_index(inplace=True)

In [ ]:
def describe_data(data = None):
    description = pd.DataFrame()
    description["Count"] = data.count()
    description["DType"] = data.dtypes
    description["# Unique"] = data.nunique()
    description["# Missing"] = data.isna().sum()
    description["% Missing"] = round(description["# Missing"] / (description["Count"]+data.isna().sum()) * 100,2)

    desc = data.describe(include="all").T
    description["Mean"] = desc["mean"]
    description["STD"] = desc["std"]
    description["Min"] = desc["min"]
    description["max"] = desc["max"]    

    return description


describe_data(raw_news_data)

,Count,DType,# Unique,# Missing,% Missing,Mean,STD,Min,max
Impression ID,156965,int64,156965,0,0.00,78483.0,45312.036839,1.0,156965.0
User ID,156965,object,50000,0,0.00,NaN,NaN,NaN,NaN
Impression Time,156965,object,125501,0,0.00,NaN,NaN,NaN,NaN
User Click History,153727,object,48868,3238,2.06,NaN,NaN,NaN,NaN
Impression News,156965,object,143617,0,0.00,NaN,NaN,NaN,NaN


In [6]:
raw_news_data.isna().sum()

News ID                         0
Category                        0
Subcategory                     0
News Title                      0
News Abstract                2666
News Url                        0
Entities in News Title          3
Entities in News Abstract       4
dtype: int64

In [46]:
news_content_based_filtering_data = raw_news_data[["Category", "Subcategory", "News Title", "News Abstract"]].fillna("")

In [47]:
news_content_based_filtering_data["content"] = news_content_based_filtering_data["Category"] + " " + news_content_based_filtering_data["Subcategory"] + " " + news_content_based_filtering_data["News Title"] + " " + news_content_based_filtering_data["News Abstract"]

In [48]:
news_content_based_filtering_data.sort_index(inplace=True)

In [49]:
news_content_based_filtering_data

,Category,Subcategory,News Title,News Abstract,content
News ID,,,,,
N10,health,weight-loss,7 Running Workouts for Weight Loss,It's all about the fartleks.,health weight-loss 7 Running Workouts for Weig...
N100,sports,baseball_mlb,Astros finally get timely hitting to get back ...,"WASHINGTON Somewhere over the eastern skies,...",sports baseball_mlb Astros finally get timely ...
N1000,sports,more_sports,"Sounders vs. Real Salt Lake: Highlights, stats...","I think we have a really good team, and a team...",sports more_sports Sounders vs. Real Salt Lake...
N10000,sports,more_sports,"The Day in Sports: Wednesday, October 30, 2019","The Day in Sports: Wednesday, October 30, 2019",sports more_sports The Day in Sports: Wednesda...
N10001,news,newscrime,Vero Beach man charged with battery in connect...,A Vero Beach man was charged with battery afte...,news newscrime Vero Beach man charged with bat...
...,...,...,...,...,...
N9993,sports,more_sports,Game Preview: Suns (1-0) try to move on withou...,,sports more_sports Game Preview: Suns (1-0) tr...
N9994,news,newsus,ECCD Celebrates Year In Review,The annual meeting of the Eastern Connecticut ...,news newsus ECCD Celebrates Year In Review The...
N9997,lifestyle,shop-holidays,30 Holiday Gift Bargains From Costco,Gifts for loved ones and holiday hosts can be ...,lifestyle shop-holidays 30 Holiday Gift Bargai...


In [ ]:
import re
import string

def clean_text(text):
    stopwords_english = stopwords.words('english')
    stemmer = WordNetLemmatizer()
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)

    text = ' '.join(word for word in text.split(' ') if word not in stopwords_english)
    text = ' '.join(stemmer.lemmatize(word) for word in text.split(" "))

    return text


In [51]:
news_content_based_filtering_data["Content Clean"] = news_content_based_filtering_data["content"].apply(clean_text)
news_content_based_filtering_data

,Category,Subcategory,News Title,News Abstract,content,Content Clean
News ID,,,,,,
N10,health,weight-loss,7 Running Workouts for Weight Loss,It's all about the fartleks.,health weight-loss 7 Running Workouts for Weig...,health weightloss running workout weight loss...
N100,sports,baseball_mlb,Astros finally get timely hitting to get back ...,"WASHINGTON Somewhere over the eastern skies,...",sports baseball_mlb Astros finally get timely ...,sport baseballmlb astros finally get timely hi...
N1000,sports,more_sports,"Sounders vs. Real Salt Lake: Highlights, stats...","I think we have a really good team, and a team...",sports more_sports Sounders vs. Real Salt Lake...,sport moresports sounder v real salt lake high...
N10000,sports,more_sports,"The Day in Sports: Wednesday, October 30, 2019","The Day in Sports: Wednesday, October 30, 2019",sports more_sports The Day in Sports: Wednesda...,sport moresports day sport wednesday october ...
N10001,news,newscrime,Vero Beach man charged with battery in connect...,A Vero Beach man was charged with battery afte...,news newscrime Vero Beach man charged with bat...,news newscrime vero beach man charged battery ...
...,...,...,...,...,...,...
N9993,sports,more_sports,Game Preview: Suns (1-0) try to move on withou...,,sports more_sports Game Preview: Suns (1-0) tr...,sport moresports game preview sun try move wi...
N9994,news,newsus,ECCD Celebrates Year In Review,The annual meeting of the Eastern Connecticut ...,news newsus ECCD Celebrates Year In Review The...,news newsus eccd celebrates year review annual...
N9997,lifestyle,shop-holidays,30 Holiday Gift Bargains From Costco,Gifts for loved ones and holiday hosts can be ...,lifestyle shop-holidays 30 Holiday Gift Bargai...,lifestyle shopholidays holiday gift bargain c...


In [13]:
nltk.download('stopwords')
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [58]:
from scipy.sparse import save_npz
import joblib


tfidf = TfidfVectorizer()
tfidf_embedding_bank = tfidf.fit_transform(news_content_based_filtering_data["Content Clean"])

save_npz("app/model_assets/tfidf_embeddings.npz", tfidf_embedding_bank)
joblib.dump(tfidf, "app/model_assets/tfidf_vectorizer.pkl")

['app/model_assets/tfidf_vectorizer.pkl']

In [59]:
tfidf_embedding_bank

<51282x56700 sparse matrix of type '<class 'numpy.float64'>'
	with 1266837 stored elements in Compressed Sparse Row format>

In [130]:
def tfidf_recommendation(news, target, embedding_bank, model, topk=10):
    content = target["Category"] + " " + target["Subcategory"] + " " + target["News Title"] + " " + target["News Abstract"]
    target = clean_text(content)
    target_vector = model.transform([target])

    similarities = cosine_similarity(embedding_bank, target_vector).flatten()
    topk_indices = similarities.argsort()[::-1][1:topk+1]

    top_simil = [
                    {
                        "News ID": news.index[i],
                        "Category": news.iloc[i]["Category"],
                        "Subcategory": news.iloc[i]["Subcategory"],
                        "News Title": news.iloc[i]["News Title"],
                        "News Abstract": news.iloc[i]["News Abstract"],
                        "Similarity": round(float(similarities[i]), 4)
                    }
                   for i in topk_indices]

    return top_simil

test = tfidf_recommendation(raw_news_data, raw_news_data.loc["N10"], tfidf_embedding_bank, tfidf, 1)
test

[{'News ID': 'N28111',
  'Category': 'health',
  'Subcategory': 'weightloss',
  'News Title': 'Why Lifting Weights Is Better Than Cardio for Weight Loss After 60',
  'News Abstract': "The more muscle mass you have, the more calories you'll burn.",
  'Similarity': 0.3543}]

In [18]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [19]:
'''
tagged_data = [TaggedDocument(words=word_tokenize(doc),
                              tags=[str(i)]) for i,
               doc in enumerate(news_content_based_filtering_data["Content Clean"])]

model = Doc2Vec(vector_size=100, dm=1,
                min_count=2,window=5,workers=4, epochs=50)

model.build_vocab(tagged_data)

model.train(tagged_data,
            total_examples=model.corpus_count,
            epochs=model.epochs)


document_vectors = [model.infer_vector(
    word_tokenize(doc.lower())) for doc in news_content_based_filtering_data["Content Clean"]]

'''

'\ntagged_data = [TaggedDocument(words=word_tokenize(doc),\n                              tags=[str(i)]) for i,\n               doc in enumerate(news_content_based_filtering_data["Content Clean"])]\n\nmodel = Doc2Vec(vector_size=100, dm=1,\n                min_count=2,window=5,workers=4, epochs=50)\n\nmodel.build_vocab(tagged_data)\n\nmodel.train(tagged_data,\n            total_examples=model.corpus_count,\n            epochs=model.epochs)\n\n\ndocument_vectors = [model.infer_vector(\n    word_tokenize(doc.lower())) for doc in news_content_based_filtering_data["Content Clean"]]\n\n'

In [ ]:
'''
def word2vec_recommendation(newsid):
    target = news_content_based_filtering_data[news_content_based_filtering_data["News ID"] == newsid]["Content Clean"].iloc[0]
    target_vector = model.infer_vector(word_tokenize(target.lower()))

    similar_docs = pd.Series(model.dv.most_similar([target_vector], topn=None), name="Simil")    
    return similar_docs
'''

In [ ]:
from sentence_transformers import SentenceTransformer
transformer = SentenceTransformer("all-MiniLM-L6-v2")
transformer_embeddings = transformer.encode(news_content_based_filtering_data["content"], show_progress_bar=True, convert_to_tensor=True)

def save_bert_embeddings(tensor, path="bert_embeddings.pt"):
    """
    Saves the BERT embeddings tensor to disk using PyTorch.
    Args:
        tensor (torch.Tensor): The embeddings to save.
        path (str): File path to save to.
    """
    torch.save(tensor, path)

In [114]:
save_bert_embeddings(transformer_embeddings, "F:/projects/Porfolio/recommender-system/app/bert_embeddiings.pt")
transformer.save("app/model_assets/bert_model")


array([1.        , 0.02544221, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [125]:
raw_news_data.loc["N10"]

Category                                                                health
Subcategory                                                        weight-loss
News Title                                  7 Running Workouts for Weight Loss
News Abstract                                     It's all about the fartleks.
News Url                         https://assets.msn.com/labs/mind/AADnDn5.html
Entities in News Title       [{"Label": "Exercise", "Type": "C", "WikidataI...
Entities in News Abstract                                                   []
Name: N10, dtype: object

In [126]:
from sentence_transformers.util import cos_sim
import numpy as np

transformer = SentenceTransformer("app/model_assets/bert_model")
bert_embeddings = torch.load("F:/projects/Porfolio/recommender-system/app/bert_embeddiings.pt")

def bert_recommendation(news, target, embedding_bank, model, topk=10):
    content = target["Category"] + " " + target["Subcategory"] + " " + target["News Title"] + " " + target["News Abstract"]
    target_vector = model.encode(content, convert_to_tensor=True)

    similarities = cos_sim(target_vector, embedding_bank)[0].cpu().numpy()

    topk_indices = similarities.argsort()[::-1][1:topk+1]  # +1 to skip identical article

    # Step 5: Build response
    top_simil = [
                    {
                        "News ID": news.index[i],
                        "Category": news.iloc[i]["Category"],
                        "Subcategory": news.iloc[i]["Subcategory"],
                        "News Title": news.iloc[i]["News Title"],
                        "News Abstract": news.iloc[i]["News Abstract"],
                        "Similarity": round(float(similarities[i]), 4)
                    }
                   for i in topk_indices]

    return top_simil

bert_recommendation(raw_news_data, raw_news_data.loc["N10"],bert_embeddings, transformer,1)

[{'News ID': 'N17337',
  'Category': 'health',
  'Subcategory': 'weightloss',
  'News Title': 'I Lost 20 Pounds in 3 Months by Combining These Popular At-Home Programs',
  'News Abstract': 'Like millions of women before me, I turned to WW to help manage my eating. Later, I started doing the BBG fitness program, which consists of three 28-minute resistance training sessions and three to four cardio sessions per week.',
  'Similarity': 0.4967}]

In [ ]:
def bert_recommendation(news, target, embedding_bank, model, topk=10):
    content = target["Category"] + " " + target["Subcategory"] + " " + target["News Title"] + " " + target["News Abstract"]
    target_vector = model.encode(content, convert_to_tensor=True)

    similarities = cos_sim(target_vector, embedding_bank)[0].cpu().numpy().round(4)

    return similarities

bert_test = bert_recommendation(raw_news_data, raw_news_data.loc["N10"],bert_embeddings, transformer, 10)

array([ 1.000e+00,  1.136e-01,  1.073e-01, ...,  6.600e-02, -7.000e-04,
        5.350e-02], dtype=float32)

In [141]:
def tfidf_recommendation(news, target, embedding_bank, model, topk=10):
    content = target["Category"] + " " + target["Subcategory"] + " " + target["News Title"] + " " + target["News Abstract"]
    target = clean_text(content)
    target_vector = model.transform([target])

    similarities = cosine_similarity(embedding_bank, target_vector).flatten()

    return similarities

test = tfidf_recommendation(raw_news_data, raw_news_data.loc["N10"], tfidf_embedding_bank, tfidf, 1)
test

array([1.        , 0.02544221, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [4]:
import pandas as pd

raw_behaviors_data = pd.read_csv("F:/projects/Porfolio/recommender-system/app/model_assets/behaviors.tsv",delimiter="\t",
                                  names=["Impression ID" , "User ID" , "Impression Time" , "User Click History" ,"Impression News"])

raw_behaviors_data

,Impression ID,User ID,Impression Time,User Click History,Impression News
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...
...,...,...,...,...,...
156960,156961,U21593,11/14/2019 10:24:05 PM,N7432 N58559 N1954 N43353 N14343 N13008 N28833...,N2235-0 N22975-0 N64037-0 N47652-0 N11378-0 N4...
156961,156962,U10123,11/13/2019 6:57:04 AM,N9803 N104 N24462 N57318 N55743 N40526 N31726 ...,N3841-0 N61571-0 N58813-0 N28213-0 N4428-0 N25...
156962,156963,U75630,11/14/2019 10:58:13 AM,N29898 N59704 N4408 N9803 N53644 N26103 N812 N...,N55913-0 N62318-0 N53515-0 N10960-0 N9135-0 N5...
156963,156964,U44625,11/13/2019 2:57:02 PM,N4118 N47297 N3164 N43295 N6056 N38747 N42973 ...,N6219-0 N3663-0 N31147-0 N58363-0 N4107-0 N457...


In [148]:
def calculate_user_item_matrix(data):
    user_item_matrix = data[["User ID", "User Click History"]].fillna("").groupby("User ID")["User Click History"].apply(lambda list : sum(list.str.split(), []))
    users_ids = user_item_matrix.index

    mlb = MultiLabelBinarizer(sparse_output=True)
    user_item_sparse_matrix = mlb.fit_transform(user_item_matrix)

    items_ids = mlb.classes_

    return users_ids, items_ids, user_item_sparse_matrix

users_ids, items_ids, user_item_sparse_matrix = calculate_user_item_matrix(raw_behaviors_data) 

In [24]:
"""
def calculate_user_sim_df(user_item_matrix, users_ids):
    user_user_similarity_matrix = cosine_similarity(user_item_matrix,dense_output=False)
    user_sim_df = pd.DataFrame.sparse.from_spmatrix(user_user_similarity_matrix, index=users_ids, columns=users_ids)
    return user_sim_df

def user_based_cf_recommendation(UserID, user_sim_df):

    top_similar_users = user_sim_df[UserID].sort_values(ascending=False).iloc[1:10].index

    recommended_items = []
    for user in top_similar_users:
        recommended_items += user_item_matrix[user]

    recommended_items = set(recommended_items)

    already_clicked = set(user_item_matrix[UserID])
    recommended_items -= already_clicked

    news_lookup = raw_news_data.set_index("News ID").to_dict(orient="index")

    results = pd.DataFrame([
        {
            "News ID": new,
            "Category": news_lookup[new]["Category"],
            "Subcategory": news_lookup[new]["Subcategory"],
            "News Title": news_lookup[new]["News Title"],
            "News Abstract": news_lookup[new]["News Abstract"],
        }
        for new in recommended_items if new in news_lookup
    ])


    return results.head(10)

user_sim_df = calculate_user_sim_df(user_item_sparse_matrix, users_ids)
user_based_cf_recommendation("U100", user_sim_df)

"""

'\ndef calculate_user_sim_df(user_item_matrix, users_ids):\n    user_user_similarity_matrix = cosine_similarity(user_item_matrix,dense_output=False)\n    user_sim_df = pd.DataFrame.sparse.from_spmatrix(user_user_similarity_matrix, index=users_ids, columns=users_ids)\n    return user_sim_df\n\ndef user_based_cf_recommendation(UserID, user_sim_df):\n\n    top_similar_users = user_sim_df[UserID].sort_values(ascending=False).iloc[1:10].index\n\n    recommended_items = []\n    for user in top_similar_users:\n        recommended_items += user_item_matrix[user]\n\n    recommended_items = set(recommended_items)\n\n    already_clicked = set(user_item_matrix[UserID])\n    recommended_items -= already_clicked\n\n    news_lookup = raw_news_data.set_index("News ID").to_dict(orient="index")\n\n    results = pd.DataFrame([\n        {\n            "News ID": new,\n            "Category": news_lookup[new]["Category"],\n            "Subcategory": news_lookup[new]["Subcategory"],\n            "News Title

In [149]:
def calculate_item_sim_df(user_item_matrix, items_ids, scaling_factor=10):
    sim_matrix = cosine_similarity(user_item_matrix.T, dense_output=False)

    co_clicks = user_item_matrix.T @ user_item_matrix

    # Log-scaled weights to reduce small co-click influence more naturally
    weights = co_clicks.copy()
    weights.data = np.clip(np.log1p(weights.data) / np.log1p(scaling_factor), 0, 1)

    sim_matrix = sim_matrix.multiply(weights)

    item_sim_df = pd.DataFrame.sparse.from_spmatrix(sim_matrix, index=items_ids, columns=items_ids)
    return item_sim_df


def item_based_cf_recommendation(newsID, item_sim_df):
    if newsID not in item_sim_df.columns:
        return pd.Series(dtype=float)
    
    similarity = item_sim_df[newsID]
    return similarity

item_sim_df = calculate_item_sim_df(user_item_sparse_matrix, items_ids)

In [151]:
item_sim_df.to_pickle("app/model_assets/item_sim_df.pkl")

In [156]:
class HybridRecommender:
    def __init__(self, news, target, embeddings, model, item_sim_df,
                 mode="bert", alpha=0.5, topk=10):
        """
        mode: "bert" or "tfidf"
        """
        self.data = news  # full sorted news data (indexed)
        self.target = target  # row of the target article
        self.embeddings = embeddings
        self.model = model
        self.item_sim_df = item_sim_df  # sparse DF
        self.alpha = alpha
        self.topk = topk
        self.mode = mode

    def recommend(self, target_id):
        content_score = self.calculate_content_score()
        cf_score = self.calculate_cf_score(target_id)
        return self.combine_scores(content_score, cf_score)

    def calculate_content_score(self):
        content = (
            self.target["Category"] + " " +
            self.target["Subcategory"] + " " +
            self.target["News Title"] + " " +
            self.target["News Abstract"]
        )

        if self.mode == "bert":
            target_vector = self.model.encode(content, convert_to_tensor=True)
            similarities = cos_sim(target_vector, self.embeddings)[0].cpu().numpy()

        elif self.mode == "tfidf":
            clean_text = clean_text(content)
            target_vector = self.model.transform([clean_text])
            similarities = cosine_similarity(self.embeddings, target_vector).flatten()

        else:
            raise ValueError("Invalid mode. Choose 'bert' or 'tfidf'.")

        return similarities

    def calculate_cf_score(self, target_id):
        # Ensure it's in index
        if target_id not in self.item_sim_df.columns:
            return pd.Series([0] * len(self.data), index=self.data.index)

        # Sparse matrix column → full Series (pandas handles sparse+dense fine)
        cf_series = self.item_sim_df[target_id]
        return cf_series.to_numpy()


    def normalize(self, score):
        # score can be Series or ndarray
        if isinstance(score, pd.Series):
            values = score.values
        else:
            values = score

        min_val = np.min(values)
        max_val = np.max(values)
        if max_val - min_val < 1e-8:
            return pd.Series([0.0] * len(values), index=self.data.index)

        norm = (values - min_val) / (max_val - min_val + 1e-8)
        return np.round(norm, 2)

    def combine_scores(self, content_score, cf_score):
        content_score = self.normalize(content_score)
        cf_score = pd.Series(self.normalize(cf_score), index=self.item_sim_df.index, name="cf_score")
        print(cf_score)

        # Attach scores to dataframe
        result_df = self.data[["Category", "Subcategory", "News Title", "News Abstract"]].copy()
        result_df["content_score"] = content_score
        result_df = pd.merge(result_df, cf_score, left_on="News ID", right_on=cf_score.index,how="left")
        result_df["cf_score"] = result_df["cf_score"].fillna(0)
        result_df["hybrid_score"] = (
            self.alpha * result_df["content_score"] +
            (1 - self.alpha) * result_df["cf_score"]
        )

        return result_df.sort_values("hybrid_score", ascending=False).head(self.topk).to_dict(orient="records")
    
recommender_ = HybridRecommender(
        news=raw_news_data,
        target=raw_news_data.loc["N10"],
        embeddings=transformer_embeddings,
        model=transformer,
        item_sim_df=item_sim_df,
        mode="bert",
        alpha=0.5,
        topk=10
    )

recommender_.recommend("N10")

N10       1.0
N100      0.0
N1000     0.0
N10000    0.0
N10001    0.0
         ... 
N9987     0.0
N9988     0.0
N9990     0.0
N9992     0.0
N9993     0.0
Name: cf_score, Length: 33195, dtype: float64


[{'News ID': 'N10',
  'Category': 'health',
  'Subcategory': 'weight-loss',
  'News Title': '7 Running Workouts for Weight Loss',
  'News Abstract': "It's all about the fartleks.",
  'content_score': 1.0,
  'cf_score': 1.0,
  'hybrid_score': 1.0},
 {'News ID': 'N23618',
  'Category': 'autos',
  'Subcategory': 'autosnews',
  'News Title': 'Fake voicemails used to hack Microsoft 365 accounts',
  'News Abstract': 'Microsoft 365 users hit by phishing campaign using voicemail scam.',
  'content_score': 0.18000000715255737,
  'cf_score': 1.0,
  'hybrid_score': 0.5900000035762787},
 {'News ID': 'N6686',
  'Category': 'finance',
  'Subcategory': 'financenews',
  'News Title': "Amazon Web Services is 'evaluating options' after Microsoft wins $10 billion JEDI contract out from under its nose",
  'News Abstract': "It's a surprise defeat to Amazon, and it's not clear what its options might be.",
  'content_score': 0.15000000596046448,
  'cf_score': 1.0,
  'hybrid_score': 0.5750000029802322},
 {'Ne

In [164]:


class HybridRecommender:
    def __init__(self, data, bert_embeddings, item_sim_df, user_item_matrix, alpha=0.5):
        self.data = data
        self.bert_embeddings = bert_embeddings
        self.item_sim_df = item_sim_df
        self.user_item_matrix = user_item_matrix
        self.alpha = alpha

    def recommend(self, target_id, topk=10):
        bert_score = self.calculate_bert_scores(target_id)
        cf_score = self.calculate_cf_scores(target_id)

        return self.combine_scores(bert_score, cf_score, topk, target_id)

    def calculate_bert_scores(self, target_id):
        query_index = self.data[self.data["News ID"] == target_id].index[0] 
        query_embedding = self.bert_embeddings[query_index]
        cos_scores = pd.Series(cos_sim(query_embedding, self.bert_embeddings)[0].cpu().numpy(), name="bert_score")

        return cos_scores
    
    def calculate_cf_scores(self, target_id):
        if target_id not in self.item_sim_df.columns:
            return pd.Series(dtype=float)
    
        similarity = self.item_sim_df[target_id]
        return similarity
    
    def normalize(self, score):
        if score.empty:
            return score
        min_score = score.min()
        max_score = score.max()
        return (score - min_score) / (max_score - min_score + 1e-8)

    def combine_scores(self, bert_score, cf_score, topk, item_id):
        bert_score = self.normalize(pd.Series(bert_score, name="bert_score"))
        cf_score = self.normalize(pd.Series(cf_score, name="cf_score"))

        hybrid_recommender_data = pd.concat([self.data.reset_index(drop=True), bert_score.reset_index(drop=True)], axis=1)
        hybrid_recommender_data = hybrid_recommender_data.merge(cf_score, how="left", left_on="News ID", right_index=True)
        hybrid_recommender_data["cf_score"] = hybrid_recommender_data["cf_score"].fillna(0)
        hybrid_recommender_data["hybrid_score"] = (
            self.alpha * hybrid_recommender_data["bert_score"] +
            (1 - self.alpha) * hybrid_recommender_data["cf_score"]
        )

        result = hybrid_recommender_data[hybrid_recommender_data["News ID"] != item_id]
        return result.sort_values("hybrid_score", ascending=False).head(topk)
        

In [129]:
class Evaluator:
    def __init__(self, recommender, user_clicks, k=10):
        self.recommender = recommender
        self.user_clicks = user_clicks
        self.k = k

    def precision_at_k(self, actual, predicted):
        if not actual:
            return 0
        
        predicted = predicted[:self.k]
        hits = len(set(actual) & set(predicted))
        return hits/self.k
    
    def recall_at_k(self, actual, predicted):
        if not actual:
            return 0
        
        predicted = predicted[:self.k]
        hits = len(set(actual) & set(predicted))
        return hits / len(actual)
    
    def ndcg_at_k(self, actual, predicted):
        predicted = predicted[:self.k]
        dcg = 0.0
        idcg = 0.0

        for idx, p in enumerate(predicted):
            if p in actual:
                dcg += 1 / np.log2(idx + 2)  # +2 because idx starts from 0

        for idx in range(min(len(actual), self.k)):
            idcg += 1 / np.log2(idx + 2)

        return dcg / idcg if idcg > 0 else 0
    
    def evaluate_user(self, user_id):
        """
        Evaluate recommendations for a single user.
        """
        actual_clicks = self.user_clicks.get(user_id, [])
        
        if not actual_clicks:
            return None  # No data for this user

        # Assume last click is the "query" article
        last_clicked = actual_clicks[-1]

        # All earlier clicks are the "ground truth" to predict
        ground_truth = actual_clicks[:-1]

        if not ground_truth:
            return None

        # Get recommendations
        recs = self.recommender.recommend(last_clicked, topk=self.k)
        recommended_news_ids = recs["News ID"].tolist()   

        # Calculate metrics
        precision = self.precision_at_k(ground_truth, recommended_news_ids)
        recall = self.recall_at_k(ground_truth, recommended_news_ids)
        ndcg = self.ndcg_at_k(ground_truth, recommended_news_ids)

        return {"precision": precision, "recall": recall, "ndcg": ndcg}
    
    def evaluate_all(self):
        """
        Evaluate across all users.
        Returns average metrics.
        """
        all_metrics = []

        for user_id in self.user_clicks.keys():
            metrics = self.evaluate_user(user_id)
            if metrics:
                all_metrics.append(metrics)

        if not all_metrics:
            return {}

        # Average the results
        avg_precision = np.mean([m["precision"] for m in all_metrics])
        avg_recall = np.mean([m["recall"] for m in all_metrics])
        avg_ndcg = np.mean([m["ndcg"] for m in all_metrics])

        return {
            "avg_precision": avg_precision,
            "avg_recall": avg_recall,
            "avg_ndcg": avg_ndcg
        }

In [125]:
def encode_new_item(news_row, bert_model):
    """Encode a single news row into a BERT embedding."""
    # Get the raw content fields
    category = str(news_row["Category"].values[0])
    subcategory = str(news_row["Subcategory"].values[0])
    title = str(news_row["News Title"].values[0])
    abstract = str(news_row["News Abstract"].values[0])
    
    text = f"{category} {subcategory} {title} {abstract}"
    text = clean_text(text)

    # Encode
    new_embedding = bert_model.encode(text, convert_to_tensor=True)

    return new_embedding

        
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)

    text = ' '.join(word for word in text.split(' ') if word not in stopwords_english)
    text = ' '.join(stemmer.lemmatize(word) for word in text.split(" "))
    return text

In [85]:
train_user_clicks = raw_behaviors_data[["User ID", "User Click History"]].fillna("").groupby("User ID")["User Click History"].apply(lambda list : sum(list.str.split(), [])).to_dict()

In [171]:
raw_news_data = pd.read_csv("F:/projects/Porfolio/recommender-system/app/model_assets/news.tsv",delimiter="\t",
                             names=["News ID" , "Category" , "Subcategory" , "News Title" ,"News Abstract", "News Url", "Entities in News Title", "Entities in News Abstract"],
                             ).sort_values("News ID")

hybrid_recommender = HybridRecommender(raw_news_data, transformer_embeddings, item_sim_df, user_item_sparse_matrix, 0.5)
bert_recommender = HybridRecommender(raw_news_data, transformer_embeddings, item_sim_df, user_item_sparse_matrix, 1)
cf_recommender = HybridRecommender(raw_news_data, transformer_embeddings, item_sim_df, user_item_sparse_matrix, 0)

In [173]:
hybrid_recommender.recommend("N10")

,News ID,Category,Subcategory,News Title,News Abstract,News Url,Entities in News Title,Entities in News Abstract,bert_score,cf_score,hybrid_score
48421,N6686,finance,financenews,Amazon Web Services is 'evaluating options' af...,"It's a surprise defeat to Amazon, and it's not...",https://assets.msn.com/labs/mind/AAJn6Fm.html,"[{""Label"": ""Amazon Web Services"", ""Type"": ""M"",...","[{""Label"": ""Amazon (company)"", ""Type"": ""O"", ""W...",0.162562,1.0,0.581281
11931,N23618,autos,autosnews,Fake voicemails used to hack Microsoft 365 acc...,Microsoft 365 users hit by phishing campaign u...,https://assets.msn.com/labs/mind/AAJGkfV.html,"[{""Label"": ""Microsoft"", ""Type"": ""O"", ""Wikidata...","[{""Label"": ""Microsoft"", ""Type"": ""O"", ""Wikidata...",0.160963,1.0,0.580481
32788,N47581,autos,autosnews,Microsoft cloud services may be hitting their ...,Azure customers are growing increasingly conce...,https://assets.msn.com/labs/mind/AAJInE2.html,"[{""Label"": ""Microsoft"", ""Type"": ""O"", ""Wikidata...","[{""Label"": ""Microsoft Azure"", ""Type"": ""W"", ""Wi...",0.159824,1.0,0.579912
155,N10174,news,newsus,3 Arrested Following Fremont Bank Robbery; I-8...,Three people were under arrest following a ban...,https://assets.msn.com/labs/mind/AAJxRrP.html,"[{""Label"": ""Fremont, California"", ""Type"": ""G"",...","[{""Label"": ""Fremont, California"", ""Type"": ""G"",...",1.000000,0.0,0.5
40798,N56787,news,newsscienceandtechnology,Slack looks to richer apps to help it fend off...,More functional and interactive apps are on th...,https://assets.msn.com/labs/mind/AAJaTib.html,"[{""Label"": ""Slack (software)"", ""Type"": ""W"", ""W...",[],0.254858,0.707107,0.480983
38622,N54308,finance,financenews,How Apple and Microsoft could blow up the stoc...,"Microsoft, Apple, Amazon, Google owner Alphabe...",https://assets.msn.com/labs/mind/AAJIi1r.html,"[{""Label"": ""Apple Inc."", ""Type"": ""O"", ""Wikidat...","[{""Label"": ""Apple Inc."", ""Type"": ""O"", ""Wikidat...",0.249733,0.707107,0.47842
36230,N51532,news,newsbusiness,Microsoft Stock at Record After Scoring Pentag...,The software icon won a 10-year cloud contract...,https://assets.msn.com/labs/mind/AAJt4ya.html,"[{""Label"": ""Microsoft"", ""Type"": ""O"", ""Wikidata...",[],0.237588,0.707107,0.472347
34479,N49536,sports,football_nfl,Seahawks QB Russell Wilson named NFC Offensive...,"For the second time this season, Seahawks quar...",https://assets.msn.com/labs/mind/AAJX0wq.html,"[{""Label"": ""Russell Wilson"", ""Type"": ""P"", ""Wik...","[{""Label"": ""Russell Wilson"", ""Type"": ""P"", ""Wik...",0.227722,0.707107,0.467415
36556,N51918,finance,finance-top-stocks,Is Microsoft (MSFT) Outperforming Other Comput...,Investors focused on the Computer and Technolo...,https://assets.msn.com/labs/mind/AAJt4Qb.html,"[{""Label"": ""Information technology"", ""Type"": ""...","[{""Label"": ""Information technology"", ""Type"": ""...",0.197361,0.707107,0.452234
327,N1037,finance,finance-companies,"CEO departures hit a new high in October, on p...",October marked the highest month on record wit...,https://assets.msn.com/labs/mind/AAJWqVs.html,[],"[{""Label"": ""Challenger, Gray & Christmas"", ""Ty...",0.237717,0.57735,0.407533


In [86]:
import random

sampled_users = random.sample(list(train_user_clicks.keys()), 500)

hybrid_evaluator = Evaluator(hybrid_recommender, {u: train_user_clicks[u] for u in sampled_users}, k=10)
hybrid_results = hybrid_evaluator.evaluate_all()
print("hybrid") 
print(hybrid_results)

bert_evaluator = Evaluator(bert_recommender, {u: train_user_clicks[u] for u in sampled_users}, k=10)
bert_results = bert_evaluator.evaluate_all()
print("bert")
print(bert_results)

cf_evaluator = Evaluator(cf_recommender, {u: train_user_clicks[u] for u in sampled_users}, k=10)
cf_results = cf_evaluator.evaluate_all()
print("cf")
print(cf_results)

hybrid
{'avg_precision': 0.009533468559837728, 'avg_recall': 0.003409228570603343, 'avg_ndcg': 0.012686944476735285}
bert
{'avg_precision': 0.004462474645030426, 'avg_recall': 0.001912114307970655, 'avg_ndcg': 0.0054532661316968965}
cf
{'avg_precision': 0.060649087221095334, 'avg_recall': 0.02704561401263207, 'avg_ndcg': 0.06691660291921447}


In [ ]:
val_behaviors_data = pd.read_csv("F:/projects/Porfolio/recommender-system/MINDsmall_dev/behaviors.tsv", delimiter="\t",names=["Impression ID" , "User ID" , "Impression Time" , "User Click History" ,"Impression News"])
val_news_data = pd.read_csv("MINDsmall_dev/news.tsv",delimiter="\t", names=["News ID" , "Category" , "Subcategory" , "News Title" ,"News Abstract", "News Url", "Entities in News Title", "Entities in News Abstract"])

val_user_clicks = val_behaviors_data[["User ID", "User Click History"]].fillna("").groupby("User ID")["User Click History"].apply(lambda list : sum(list.str.split(), [])).to_dict()

In [126]:
full_data = raw_news_data.copy()

for new in val_news_data["News ID"]:
    if new not in raw_news_data["News ID"].values:
        news_row = val_news_data[val_news_data["News ID"] == new]
        new_embedding = encode_new_item(news_row, transformer)
        transformer_embeddings = torch.cat([transformer_embeddings, new_embedding.unsqueeze(0)], dim=0)
        full_data = pd.concat([full_data, news_row], axis=0)


In [127]:
hybrid_recommender = HybridRecommender(full_data, transformer_embeddings, item_sim_df, user_item_sparse_matrix, 0.5)
bert_recommender = HybridRecommender(full_data, transformer_embeddings, item_sim_df, user_item_sparse_matrix, 1)
cf_recommender = HybridRecommender(full_data, transformer_embeddings, item_sim_df, user_item_sparse_matrix, 0)

In [103]:
val_user_clicks = val_behaviors_data[["User ID", "User Click History"]].fillna("").groupby("User ID")["User Click History"].apply(lambda list : sum(list.str.split(), [])).to_dict()

In [131]:
import random

sampled_users = random.sample(list(val_user_clicks.keys()), 500)

hybrid_evaluator = Evaluator(hybrid_recommender, {u: val_user_clicks[u] for u in sampled_users}, k=10)
hybrid_results = hybrid_evaluator.evaluate_all()
print("hybrid") 
print(hybrid_results)

bert_evaluator = Evaluator(bert_recommender, {u: val_user_clicks[u] for u in sampled_users}, k=10)
bert_results = bert_evaluator.evaluate_all()
print("bert")
print(bert_results)

cf_evaluator = Evaluator(cf_recommender, {u: val_user_clicks[u] for u in sampled_users}, k=10)
cf_results = cf_evaluator.evaluate_all()
print("cf")
print(cf_results)

hybrid
{'avg_precision': 0.007352941176470588, 'avg_recall': 0.0035177875580217227, 'avg_ndcg': 0.007951074163418375}
bert
{'avg_precision': 0.004201680672268907, 'avg_recall': 0.0022088527142572847, 'avg_ndcg': 0.004196791916260836}
cf
{'avg_precision': 0.0411764705882353, 'avg_recall': 0.01954647568460052, 'avg_ndcg': 0.04538440543205669}


In [132]:
hybrid_recommender = HybridRecommender(full_data, transformer_embeddings, item_sim_df, user_item_sparse_matrix, 0.3)
hybrid_evaluator = Evaluator(hybrid_recommender, {u: val_user_clicks[u] for u in sampled_users}, k=10)
hybrid_results = hybrid_evaluator.evaluate_all()
print("hybrid alpha = 0.3") 
print(hybrid_results)

hybrid_recommender = HybridRecommender(full_data, transformer_embeddings, item_sim_df, user_item_sparse_matrix, 0.2)
hybrid_evaluator = Evaluator(hybrid_recommender, {u: val_user_clicks[u] for u in sampled_users}, k=10)
hybrid_results = hybrid_evaluator.evaluate_all()
print("hybrid alpha = 0.2") 
print(hybrid_results)


hybrid_recommender = HybridRecommender(full_data, transformer_embeddings, item_sim_df, user_item_sparse_matrix, 0.1)
hybrid_evaluator = Evaluator(hybrid_recommender, {u: val_user_clicks[u] for u in sampled_users}, k=10)
hybrid_results = hybrid_evaluator.evaluate_all()
print("hybrid alpha = 0.1") 
print(hybrid_results)

hybrid alpha = 0.3
{'avg_precision': 0.01764705882352941, 'avg_recall': 0.008728909647719392, 'avg_ndcg': 0.020512831236352707}
hybrid alpha = 0.2
{'avg_precision': 0.027941176470588237, 'avg_recall': 0.014157857537686789, 'avg_ndcg': 0.03160128613943444}
hybrid alpha = 0.1
{'avg_precision': 0.03802521008403362, 'avg_recall': 0.017189555749489254, 'avg_ndcg': 0.04296906855847849}
